In [2]:
#imports

import requests
import lxml
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# Grab html content
response = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(response.content, features='html.parser')
tbl = str(soup.find('table')) #our table is at the first ocurrence of 'table' element

In [4]:
# Transofrm raw HTML string into Pandas Dataframe object
df = pd.read_html(tbl)
df = df[0] #grab the first element since Pandas.read_html always either returns a list of DataFrames or bails

In [5]:
# Data Prepping

# Rename columns
df.columns = ['Postalcode', 'Borough', 'Neighbourhood']
df = df[df['Borough'] != 'Not assigned'] # weed out 'Not assigned' Borough entries

# Assign Borough names on 'Not assigned' Neighbourhoods
for i in range(0, df.shape[0]):
    if df.iloc[i][2] == 'Not assigned':
        df.iloc[i][2] = df.iloc[i][1]
        i = i + 1

# Group Similar Postal codes and concatting Neighbourhoods with ", "
df = df.groupby(['Postalcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [6]:
# In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.
print(df.shape)

(103, 3)


In [7]:
# part 2
# adding geocodes

#imports
import numpy as np
import pandas as pd
!pip install geocoder
import geocoder

jupyter-console 6.0.0 has requirement prompt-toolkit<2.1.0,>=2.0.0, but you'll have prompt-toolkit 1.0.16 which is incompatible.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
# postal code to coordinates function
def get_geocodes(postal_code):
    latlong_coords = None
    # loop until you get the coordinates
    while(latlong_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        latlong_coords = g.latlng
    return latlong_coords

In [9]:
# grab our data's coordinates through postal codes and the function we've created

coordinates = []
postalcodes = df['Postalcode'].tolist()
for postalcode in postalcodes:
    coordinates.append(get_geocodes(postalcode))

In [10]:
df_coords = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df

    Postalcode           Borough  \
0          M1B       Scarborough   
1          M1C       Scarborough   
2          M1E       Scarborough   
3          M1G       Scarborough   
4          M1H       Scarborough   
5          M1J       Scarborough   
6          M1K       Scarborough   
7          M1L       Scarborough   
8          M1M       Scarborough   
9          M1N       Scarborough   
10         M1P       Scarborough   
11         M1R       Scarborough   
12         M1S       Scarborough   
13         M1T       Scarborough   
14         M1V       Scarborough   
15         M1W       Scarborough   
16         M1X       Scarborough   
17         M2H        North York   
18         M2J        North York   
19         M2K        North York   
20         M2L        North York   
21         M2M        North York   
22         M2N        North York   
23         M2P        North York   
24         M2R        North York   
25         M3A        North York   
26         M3B        North 

In [11]:
df

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743125,-79.231750
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726245,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713133,-79.285055
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696665,-79.260163
